In [ ]:
!pip install dash dash-mantine-components

In [ ]:
from google.cloud import vision_v1
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State
import pandas as pd
import plotly.express as px
import dash_mantine_components as dmc
from PIL import Image
import base64
import time
import io

In [ ]:
PROJECT_ID = "-------"
LOCATION_ID = "us-east1"
PRODUCT_SET_ID = "product_set10"
PARENT=f"projects/{PROJECT_ID}/locations/{LOCATION_ID}"

In [ ]:
image_annotator_client = vision_v1.ImageAnnotatorClient()

In [ ]:

# Search products similar to the image.
def search_by_image(image):
    
    new_image = base64.b64decode(image.split(',')[1])
        
    response = image_annotator_client.product_search(
    
        vision_v1.Image(content = new_image),

        image_context = {
            "product_search_params": {
                "product_categories": ["apparel-v2"],
                "product_set": f"projects/{PROJECT_ID}/locations/{LOCATION_ID}/productSets/{PRODUCT_SET_ID}"
            }
        },

        max_results = 10
    )
    results = response.product_search_results.results
    return results

In [ ]:
def parse_contents(contents, filename, date):

    start = time.time()
    results = search_by_image(contents)
    response_time = time.time() - start
    
    children = []
    
    for result in results:
        
        prod_id = result.product.name[result.product.name.rindex('/')+1:]
        img_link = "https://{}".format(prod_id)
        prod_link = "https://{}".format(prod_id)
        
        children.append(
            html.Div([
                dmc.Image(src=img_link, width = 200),
                dmc.Anchor("SKU #{}".format(prod_id), href = prod_link),
                #dmc.Text(result.score)
                
            ])
    )
    return html.Div(
        [
            dmc.Center(html.Img(src=contents, style={'height':'10%', 'width':'10%'})),
            dmc.Space(h=10),
            dmc.Divider(variant="dotted"),
            dmc.Space(h=10),
            dmc.Center(dmc.Title("Products Found (response time {} sec):".format(round(response_time, 2)), color="blue", size="h4")),
            dmc.Space(h=10),
            dmc.Center(
                dmc.SimpleGrid(
                    cols=5,
                    spacing="lg",
                    children = children
                )
            )
        ])


In [ ]:
@callback(Output('output-image-upload', 'children'),
              Input('upload-image', 'contents'),
              State('upload-image', 'filename'),
              State('upload-image', 'last_modified'))
def update_output(contents, filename, last_modified):
    if contents is not None:
        children = [parse_contents(contents, filename, last_modified)]
        return children
    else:
        return list()



In [ ]:
# Initialize the app - incorporate a Dash Mantine theme

app = Dash(
    __name__, 
    external_stylesheets=[dmc.theme.DEFAULT_COLORS],
    serve_locally = False, 
    requests_pathname_prefix="/proxy/8050"
)

app.layout = dmc.Container([

    dmc.Space(h=30),

    dmc.Center(
        html.Div([
            dmc.Image(src=app.get_asset_url('-----.png'),
                      width=300, 
                      #height=100, 
                      withPlaceholder=True),
            dmc.Space(h=20),
            dmc.Title('Improving Search with Vision Product Search API', color="blue", size="h2"),
            dmc.Text('This demo consists of receiving a photo from the end user and using it to find candidates in 10 thousand downloaded photos of real --- products indexed using the Vision Product Search API.', size="md"),
            dmc.Text('The expected response time is 300ms', size="md"),
            dmc.Text('Questions? ----', size="md"),
            dmc.Space(h=10),
            dmc.Divider(variant="solid")
        ])
    ),

     dmc.Center(
        html.Div([
            dmc.Space(h=30),
            dcc.Upload(id='upload-image', children=dmc.Button('Search with an image. Drag & Drop or Browse'), multiple=False),
        ])
     ),

    dmc.Space(h=30),
    
    html.Div(id = 'output-image-upload')

    ], 
    fluid=True)

app.run(jupyter_mode="external", 
        jupyter_server_url="https://--------------dot-us-central1.notebooks.googleusercontent.com")

